In [ ]:
# -*- coding: utf-8 -*-

!pip install matplotlib
!pip install networkx
!pip install numpy
!pip install haversine
!pip install cartopy
!pip install geopandas
!pip install datetime
!pip install openpyxl
!pip install ortools 

import os
import numpy as np
import networkx as nx
import ortools
from ortools.linear_solver import pywraplp
from haversine import haversine
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook

# Repository klonen, falls es noch nicht vorhanden ist

repo_url = "https://github.com/rim-the-optimizer/Pricing.git"
repo_name = "Pricing"

if not os.path.exists(repo_name):
    !git clone {repo_url}
    print("✅ Repository wurde geklont.")
else:
    print("✅ Repository existiert bereits.")

# In den Repository-Ordner wechseln
os.chdir(repo_name)


#paths of Berlin instances

data_folder = os.path.join(os.getcwd(), "data_Berlin")
data1 = os.path.join(data_folder, "TWD.csv")
data2 = os.path.join(data_folder, "districts.csv")

M = 10000
time_format_output='%H:%M'


#model


"""
def pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M):
    
    global mod
    mod = gp.Model("Pricing")

    # Decision variables
    x = mod.addVars(E, H, vtype=GRB.BINARY, name="x")                          # Binary variable for matching status; =1 if node i is matched with courier h to be connected to node j
    S = mod.addVars(V, H, vtype=GRB.INTEGER,lb=0, name="S")                    # Non negative INT variable for time of courier h starting at node i in V 
    L = mod.addVars(V, H, vtype=GRB.INTEGER,lb=0, name="L")                    # Non negative INT variable for upper bound of number of parcels that a courier h is carrying after serving node i 
    z = mod.addVars(A, vtype=GRB.BINARY, name="z")                             # Binary variable, =1 if request i is placed in the request bank, 0 otherwise
    p = mod.addVars(E, H, vtype=GRB.CONTINUOUS, lb=0, name="p")                # Shipping price per km for sender
    c = mod.addVars(E, H, vtype=GRB.CONTINUOUS, lb=0, name="c")                # Compensation rate per km for courier
   
    # Objective function
    mod.setObjective(quicksum(p[i,j,h]*d2[i]-c[i, j, h]*(d[i,j]) for i,j in E for h in H if j!=i), gp.GRB.MAXIMIZE) # (39)
   
    # Constraints
    mod.addConstrs((p[i, j, h] <= WTP[i]/d2[i] + (1-x[i, j, h])*M for h in H for i in A for j in pickup_delivery  if i!=j), name='Price_not_greater_than_WTP(02)/(28)')
    mod.addConstrs((c[i, j, h] >= ETP[h]*x[i, j, h] for h in H for i in origin_pickup_delivery[h] for j in pickup_delivery  if i!=j), name='Compensation_not_less_than_ETP(03)/(29)')
    mod.addConstrs((p[i, j, h] >= c[i, j, h] for h in H for i in A for j in pickup_delivery if i!=j), name="Price_bigger_Compensation(04)")
    mod.addConstrs((quicksum(x[i, j, h] for j in pickup_delivery for h in H if j!=i) + z[i] == 1 for i in A), name="i_in_requestbank_or_picked_up(05)")
    mod.addConstrs((quicksum(x[i, j, h] for j in Vh[h] if i!=j ) - quicksum(x[j, n + i, h] for j in Vh[h] if j!=n+i) == 0 for h in H for i in A ), name="matched_picked_delivered_same_courier(06)")
    mod.addConstrs((quicksum(x[origin[h], j, h]  for j in pickup_destination[h] ) == 1 for h in H), name="courier_starts_origin_to_pickup_or_destination(07)")
    mod.addConstrs((quicksum(x[i, tau_[h], h] for i in delivery_origin[h] if i!=tau_[h]) == 1 for h in H), name="courier_arrives_destination_from_delivery_or_origin(08)")
    mod.addConstrs((quicksum(x[i, j, h] for i in V if i!=j) - quicksum(x[j, i, h] for i in V  if i!=j) == 0 for j in pickup_delivery for h in H), name="courier_arrives_leaves_droppoff(09)")
    mod.addConstrs((S[i, h] + s[i] + t[i, j] <= S[j, h] + (1-x[i, j, h])*M for (i, j) in E for h in H), name='courier_follows_matched_paths(10)/(34)')
    mod.addConstrs(( S[i, h] == [a[i], b[i]] for i in V for h in H),name="pickup_deliver_timewindow(11)")
    mod.addConstrs((S[i, h] <= S[n+i, h] for i in A for h in H), name="pickup_before_delivery(12")
    mod.addConstrs((L[i, h] + l[j] <= L[j, h]+ (1-x[i , j, h])*M for (i,j) in E for h in H ), name='capacity_for_next_loading(13)/(35)')
    mod.addConstrs((L[i, h] <= R[h] for i in V for h in H ), name="capacity_constraint(14)")
    mod.addConstrs((L[origin[h], h] == 0 for h in H ), name="origin_destination_empty_load(15)(1)")
    mod.addConstrs((L[destination[h], h] == 0 for h in H), name="origin_destination_empty_load(15)(2)")
    # additional constraints
    mod.addConstrs((quicksum(x[i, j, h] for h in H for j in V if i!=j)==1 for i in tau[1:] ), name="additional_constr1")
    mod.addConstr((quicksum(x[i, j, h] for i in tau_[1:] for h in H for j in V if i!=j)==0 ), name="additional_constr2")
    mod.addConstrs((quicksum(x[i, j, h] for h in H for i in V if i!=j) == 1 for j in tau_[1:]  ), name="additional_constr3)")
    
    # Linearization constraints
    mod.addConstrs((p[i, j, h] <= M*x[i, j, h] for i, j in E for h in H if i!=j), name="bigM_pricing(23)")
    mod.addConstrs((c[i, j, h] <= M*x[i, j, h] for (i, j) in E for h in H), name="bigM_compensation(24)")
    mod.addConstrs((p[i, j, h]*x[i, j, h]==p[i, j, h] for (i, j) in E  for h in H if i!=j), name='linearization(25)')
    mod.addConstrs((c[i, j, h]*x[i, j, h]==c[i, j, h] for (i, j) in E for h in H ),name='linearization2(26')    
  
    # Optimize the mod
    mod.setParam('TimeLimit',60*60)
    mod.optimize()
    
    mod.write('IPIC.lp')
    mod.Params.OutputFlag=1
        
    
   
    if mod.status == GRB.OPTIMAL or 9:
        
        optimal_x = mod.getAttr("x", x)
        optimal_S = mod.getAttr("x", S)
        optimal_L = mod.getAttr("x", L)
        optimal_z = mod.getAttr("x", z)
        optimal_p = mod.getAttr("x", p)
        optimal_c = mod.getAttr("x", c)

        OF_p = mod.objVal
        Runtime1=mod.Runtime 
        Optimalität=mod.MIPGap
        return OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z,Runtime1, Optimalität
    else:
        return None"""
def pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M):
    
    # Initialisieren des OR-Tools Solvers (MIP-Optimierung)
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        print("Solver konnte nicht erstellt werden!")
        return None

    # Variablen definieren
    x = {}
    for (i, j) in A:
        for h in H:
            x[i, j, h] = solver.IntVar(0, 1, f'x[{i},{j},{h}]')

    S = {}
    for i in V:
        for h in H:
            S[i, h] = solver.IntVar(0, solver.infinity(), f'S[{i},{h}]')

    L = {}
    for i in V:
        for h in H:
            L[i, h] = solver.IntVar(0, solver.infinity(), f'L[{i},{h}]')

    z = {}
    for i in A:
        z[i] = solver.IntVar(0, 1, f'z[{i}]')

    p = {}
    for (i, j) in E:
        for h in H:
            p[i, j, h] = solver.NumVar(0, solver.infinity(), f'p[{i},{j},{h}]')

    c = {}
    for (i, j) in E:
        for h in H:
            c[i, j, h] = solver.NumVar(0, solver.infinity(), f'c[{i},{j},{h}]')

    # Zielfunktion: Maximiere den Profit (Versandpreis - Kosten)
    solver.Maximize(
        solver.Sum(p[i, j, h] * d2[i] - c[i, j, h] * d[i, j] for (i, j) in E for h in H if i != j)
    )

    # Constraints
    for h in H:
        for i in A:
            for j in V:
                if i != j:
                    solver.Add(p[i, j, h] <= WTP[i] / d2[i] + (1 - x[i, j, h]) * M)

    for h in H:
        for i in A:
            for j in V:
                if i != j:
                    solver.Add(c[i, j, h] >= ETP[h] * x[i, j, h])

    for h in H:
        for i in A:
            for j in V:
                if i != j:
                    solver.Add(p[i, j, h] >= c[i, j, h])

    for i in A:
        solver.Add(solver.Sum(x[i, j, h] for j in V for h in H if i != j) + z[i] == 1)

    for h in H:
        for i in A:
            solver.Add(
                solver.Sum(x[i, j, h] for j in V if i != j) - solver.Sum(x[j, i, h] for j in V if i != j) == 0
            )

    for h in H:
        solver.Add(solver.Sum(x[i, j, h] for j in V) == 1)

    for h in H:
        solver.Add(solver.Sum(x[i, j, h] for i in V if i != j) == 1)

    for (i, j) in E:
        for h in H:
            solver.Add(S[i, h] + s[i] + t[i, j] <= S[j, h] + (1 - x[i, j, h]) * M)

    for i in V:
        for h in H:
            solver.Add(S[i, h] >= a[i])
            solver.Add(S[i, h] <= b[i])

    for i in A:
        for h in H:
            solver.Add(S[i, h] <= S[n + i, h])

    for (i, j) in E:
        for h in H:
            solver.Add(L[i, h] + l[j] <= L[j, h] + (1 - x[i, j, h]) * M)

    for i in V:
        for h in H:
            solver.Add(L[i, h] <= R[h])

    for h in H:
        solver.Add(L[0, h] == 0)
        solver.Add(L[n, h] == 0)

    # Linearization Constraints
    for (i, j) in E:
        for h in H:
            solver.Add(p[i, j, h] <= M * x[i, j, h])
            solver.Add(c[i, j, h] <= M * x[i, j, h])

    # Modell lösen
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        optimal_x = {(i, j, h): x[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_S = {(i, h): S[i, h].solution_value() for i in V for h in H}
        optimal_L = {(i, h): L[i, h].solution_value() for i in V for h in H}
        optimal_z = {i: z[i].solution_value() for i in A}
        optimal_p = {(i, j, h): p[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_c = {(i, j, h): c[i, j, h].solution_value() for (i, j) in E for h in H}

        OF_p = solver.Objective().Value()
        Runtime1 = solver.WallTime()
        
        return OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1
    else:
        return None


# data from instance
def read_data_create_sets(data1):
    delimiter=';'
    df=pd.read_csv(data1, delimiter=delimiter, header=0, parse_dates=['Time a', 'Time b'], date_parser=pd.to_datetime)
    dfdistricts=pd.read_csv(data2, delimiter=delimiter, header=0)
    m=int(df.at[0,"m"])
    n=int(df.at[0,"n"])
    if type(df.at[0,"WTP_Faktor"])==np.float64:
        WTP_Faktor=df.at[0,"WTP_Faktor"]
    if type(df.at[0,"WTP_Faktor"])== str:
        WTP_Faktor=df.loc[0:0,"WTP_Faktor"].str.replace(',','.').astype(float)[0]
    if type(df.at[0,"ETP_Faktor"])==np.float64:
        ETP_Faktor=df.at[0,"ETP_Faktor"]
    if type(df.at[0,"ETP_Faktor"])== str:
        ETP_Faktor=df.loc[0:0,"ETP_Faktor"].str.replace(',','.').astype(float)[0]
    Speed=int(df.at[0,"Speed"])
    start_rowpu=0                                                        
    start_rowdel= start_rowpu + n                                          
    start_rowdriver= start_rowdel + n
    A=df.loc[start_rowpu:start_rowdel-1,'i'].astype(int).tolist()
    A_=df.loc[start_rowdel:start_rowdriver-1,'i'].astype(int).tolist()
    AllX=df.loc[start_rowpu:start_rowdriver+2*m-1,'Long'].str.replace(',','.').astype(float).tolist()
    AllX.insert(0,0.0)
    AllY=df.loc[start_rowpu:start_rowdriver+2*m-1,'Lat'].str.replace(',','.').astype(float).tolist()
    AllY.insert(0,0.0)
    R2=df.loc[start_rowdriver:start_rowdriver+m-1,'R'].astype(int).tolist()
    R2.insert(0,0)
    district_names={idx: value for idx, value in enumerate (dfdistricts.loc[0:14,'Name'].astype(str), start=0)}
    district_coordx=dfdistricts.loc[start_rowpu:14,'Lat'].str.replace(',','.').astype(float).tolist()
    district_coordy=dfdistricts.loc[start_rowpu:14,'Long'].str.replace(',','.').astype(float).tolist()
    l={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'l'].astype(int), start=1)}
    a={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'a'].astype(int), start=1)}
    b={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'b'].astype(int), start=1)}
    s={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'s'].astype(int), start=1)}
    H = list(range(1, m + 1)) 
    TW_LB={idx: value.strftime(time_format_output) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'Time a'], start=1)}
    TW_UB={idx: value.strftime(time_format_output) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'Time b'], start=1)}
    R={h:R2[h] for h in H}
    tau=[2*n+h for h in H]                                                     # origin nodes of couriers
    tau.insert(0,0)
    tau_=[2*n + m +h for h in H]                                               # destination nodes of couriers
    tau_.insert(0,0)
    ZF=[b[i+m]-a[i] for i in tau[1:]]
    V= A + A_ +tau[1:] + tau_[1:]     
    pickup_delivery= A+A_                                                      # all pickup and delivery nodes
    destination={(h): tau_[h] for h in H}                                      # Set of destinations in dependence of driver h
    origin={(h): tau[h] for h in H}                                            # Set of origins in dependence of driver h
    pickup_destination= {h: A+[destination[h]] for h in H}                     # Unificated set of pickups and destinations in dependence of driver h    
    delivery_origin={h: [origin[h]]+A_ for h in H}                             # Unificated set of origins and drop-offs in dependence of driver h    
    delivery_destination={h: A_+[destination[h]] for h in H}
    Vh={(h): [origin[h]]+A+A_+[destination[h]] for h in H }                    # Set of all nodes in dependance of driver h
    origin_pickup_delivery={h: [origin[h]]+A+A_ for h in H}
    pickup_delivery_destination={h: pickup_delivery+[destination[h]] for h in H}
    origin_delivery_destination=A_+tau[1:]+tau_[1:] 
    E=[(i, j) for i in V for j in V if i!=j ]                                  # All edges except edges from one node to the same node back
    E1={h:[(i,j) for i in Vh[h] for j in Vh[h] if i!=j] for h in H}
    d={(i,j):haversine((AllX[i],AllY[i]),(AllX[j],AllY[j])) for i,j in E}      # travel distance for link i-->j
    d2={i:haversine((AllX[i],AllY[i]),(AllX[i+n],AllY[i+n])) for i in A}     
    d21={i:0 for i in origin_delivery_destination}
    d2.update(d21)
    t={(i,j):Speed*d[i,j] for i,j in E}
    np.random.seed(10)
    ETPa =0.5                                                 
    ETPb = 1
    ETP={h:round(np.random.uniform(ETPa,ETPb)*ETP_Faktor,2) for h in H}
    np.random.seed(10)
    WTPa = 5
    WTPb=10
    WTP={i:round(np.random.uniform(WTPa,WTPb)*WTP_Faktor,2)  for i in A} 
    
    return WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ETP, WTP, ZF, TW_LB, TW_UB, destination, origin, pickup_destination, delivery_origin, delivery_destination, Vh, origin_pickup_delivery, pickup_delivery_destination, E, E1, d, d2, t, m, n, A, A_, a, b, l, s, R2, AllX, AllY, Speed, district_names, district_coordx, district_coordy 
all_data=read_data_create_sets(data1)
WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ETP, WTP, ZF, TW_LB, TW_UB, destination, origin, pickup_destination, delivery_origin, delivery_destination, Vh, origin_pickup_delivery, pickup_delivery_destination, E, E1, d, d2, t, m, n, A, A_, a, b, l, s, R2, AllX, AllY, Speed, district_names, district_coordx, district_coordy  = all_data

#

result = pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M)# running the function for optimization model
if result is not None:                                                         # print results when optimization successful  
    OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1,Optimalität = result   
else:
    print("Optimization failed.")

#extract variables
def results():
    active_arcs=[(i, j, h) for i, j in E for h in H if optimal_x[i, j, h]>0.99] 
    active_prices=[(i, j, h) for i, j in E for h in H if optimal_p[i, j, h]>0.99]
    active_costs=[(i, j, h) for i, j in E for h in H if optimal_c[i, j, h]>0.099]   
    active_load=[(i, h) for i in V for h in H  if optimal_L[i, h]>0.99] 
    active_z=[i for i in A if optimal_z[i]>0.99] 
    active_Start=[(i, h) for i in V for h in H if optimal_S[i, h]>0.99]
    td=quicksum(optimal_x[i, j, h]*d[i,j] for h in H for i in origin_pickup_delivery[h] for j in pickup_delivery  if i!=j)
    Fahrerstopps={h:sum(optimal_x[i,j,h] for i,j in E) for h in H}
    Multistop_touren={h:Fahrerstopps[h] for h in H}
    Touren={h:[(i,j) for i,j in E if optimal_x[i,j,h]>0.99 ] for h in H}
    for h in H:
        if Fahrerstopps[h]>=2:
            Multistop_touren[h]=(Fahrerstopps[h]-1)/2
        else:
            del Multistop_touren[h]
            del Touren[h]
    active_driver=[h for h in Touren if Touren[h]!=[] ]
    possible_matches= len(A)
    matches=len(A)-quicksum(optimal_z[i] for i in A)
    used_driver=quicksum(optimal_x[i,j,h] for i in A_ for j in tau_[1:] for h in H)
    active_arcs2D=list()
    for a in active_arcs:
        active_arcs2D.extend([a[:2]])
    active=list()
    for (i,j) in active_arcs2D:
        if i in tau[1:] and j in tau_[1:]:
            pass
        else:
            active.append((i,j))
    chosen_c={(i,j,h):optimal_c[i, j, h] for i,j in E for h in H if optimal_c[i, j, h]>0.099}
    chosen_cost={h:chosen_c[i,j,h] for i,j,h in chosen_c}
    chosen_p={(i):round(optimal_p[i, j, h],2) for i,j in E for h in H if optimal_p[i, j, h]>0.099}
    opt_request_p={i:round(chosen_p[i]*d2[i],2) for i in chosen_p}
    opt_driver_c={(h):round(chosen_c[i,j,h],2) for i,j,h in chosen_c}
    Einnahmen=round(sum(optimal_p[i,j,h]*d2[i] for i,j in E for h in H),2)
    Einnahmen_je_Tour={h:round(sum(optimal_p[i,j,h]*d2[i] for i,j in E ),2)for h in H}
    Ausgaben=round(sum(optimal_c[i,j,h]*d[i,j] for i,j in E for h in H),2)
    Ausgaben_je_Tour={h:round(sum(optimal_c[i,j,h]*d[i,j] for i,j in E ),2)for h in H}
    Marge_je_Tour={h:Einnahmen_je_Tour[h]-Ausgaben_je_Tour[h] for h in active_driver}
    aktive_Auftraege=[i for i in chosen_p]
    Einnahmen_je_Auftrag={i:d2[i]*chosen_p[i] for i in aktive_Auftraege}
    Fahrerdistanz_je_Auftrag={i:d[i,j]+d[a,i] for i in aktive_Auftraege for j in pickup_delivery for h in H for a in origin_pickup_delivery[h] if (i,j) in active and (a,i) in active }	
    Ausgaben_je_Auftrag={i:Fahrerdistanz_je_Auftrag[i]*chosen_c[i,j,h] for i,j,h in chosen_c if i in aktive_Auftraege}
    Marge_je_Auftrag={i:(Einnahmen_je_Auftrag[i]-Ausgaben_je_Auftrag[i]) for i in chosen_p} 
    Kantenkosten={(i,j):chosen_c[i,j,h] for h in active_driver for (i,j) in Touren[h] if j not in tau_[1:]}
    print("total distance from origin to dropoff=", td)
    print("active_arcs (x):", active_arcs)
    print("active_z (z):", active_z)
    print("Anzahl Aufträge je Fahrer:", Multistop_touren)
    print("Anzahl möglicher matches=",possible_matches)
    print("Anzahl erfolgreicher matches=", matches)
    print("Anzahl genutzter Fahrer:", used_driver)
    print("Folgende Fahrer werden eingesetzt:", active_driver)
    print("ZFW (Marge Platform Provider in €):", round(OF_p,2))
    print("Marge je Auftrag:", [Marge_je_Auftrag])
    for i in chosen_p:
        print("WTP bei Auftrag", [i], "=", WTP[i], "gewählter Preis für Auftrag", [i], "=", opt_request_p[i], "gewählter Preis je km =", round(chosen_p[i],2) )
    for h in active_driver:
        print("ETP for driver", [h], "=", ETP[h], "choosen cost per km =", opt_driver_c[h] )
    print("Einnahmen=", Einnahmen, "Ausgaben=", Ausgaben)
    return active_arcs, active_prices, active_costs, active_load, active_z, active_Start, Touren, active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, opt_request_p, opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, matches, Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege
result2 = results()
active_arcs, active_prices, active_costs, active_load, active_z, active_Start, Touren, active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, opt_request_p, opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, matches, Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege = result2

def variables_part2():
    
    active_nodes=[]
    for a in active:
        active_nodes.extend(a[:1])
        active_nodes.extend(a[1:])
    active_nodes=list(dict.fromkeys(active_nodes))
    Touren_neu={h:(tuple) for h in Touren}
    for h in Touren:
        edgars=[(i,j) for (i,j) in Touren[h]]
        GTouren = nx.DiGraph()
        GTouren.add_edges_from(edgars)
        longest_path = nx.dag_longest_path(GTouren)
        graph_after = list(zip(longest_path[:-1], longest_path[1:]))
        Touren_neu[h]=graph_after
    Touren_sorted={h:list() for h in Touren_neu}
    for h in Touren_neu:
        liste=[]
        for a in Touren_neu[h]:
            list(a)
            liste.append(a[:1])
        liste.append(a[1:])    
        liste=[i[0] for i in liste]
        Touren_sorted[h]=liste
    opt_S_driver={(i,h):optimal_S[i,h]  for h in Touren_sorted for i in  Touren_sorted[h]}
    Starting_times={i:int(optimal_S[i,h]) for h in Touren_sorted for i in Touren_sorted[h] }
    Starting_times_formatted={}
    for key, value in Starting_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Starting_times_formatted[key] = time_str
    activitis=list()
    for h in H:
        if h not in active_driver:
            not_driving=h*-1
            activitis.append(not_driving)
        else:
            activitis.append(h)
    activitis.insert(0,0)
    used_origins={h:origin[h] for h in origin if h in active_driver}
    TW_LB_driver={h: str for h in used_origins}
    for h,i in used_origins.items():
        TW_LB_driver[h]=TW_LB[i]
    used_destinations={h:destination[h] for h in destination if h in active_driver}
    TW_UB_driver={h: str for h in used_destinations}
    for h,i in used_destinations.items():
        TW_UB_driver[h]=TW_UB[i]
    active_t={i:t[i,j] for i,j in active}
    print("Tour je Fahrer:", Touren_sorted)
    used_ETP={h:ETP[h] for h in active_driver}
    used_WTP={i:WTP[i] for i in active_nodes if i in A}
    Auftrag_Fahrer={node:driver for driver in Touren_sorted for node in Touren_sorted[driver]}
    Senders_surplus=round(sum(WTP[i] for i in aktive_Auftraege)-Einnahmen,2) 
    Couriers_surplus=round(Ausgaben-sum(ETP[h]*d[i,j]*optimal_x[i,j,h] for i,j in E for h in H if i in origin_pickup_delivery[h] and j in pickup_delivery),2)
    Umweg={(h): d[Touren_sorted[h][-2],destination[h]] for h in Touren_sorted }
    Umwege_summe=round(sum(Umweg[h]for h in Umweg),2)
    ges_km_je_Tour= {h: round(sum(d[i] for i in Touren_neu[h]),2) for h in Touren_neu}
    origin_dest_km={h: round(d[origin[h],destination[h]],2) for h in H}
    Umweg_ges_km=round(Umwege_summe/sum(ges_km_je_Tour[h] for h in Touren_sorted ),2)
    Umweg_Tour_Verhältnis={h:round(Umweg[h]/ges_km_je_Tour[h],2) for h in Touren_sorted}
    km_Auftrag={i:d[i,i+n] for i in A}
    Umweg_je_Tour={h: Umweg[h]/origin_dest_km[h] for h in Touren_sorted}
    ave_Umweg_je_Tour=np.mean(list(Umweg_je_Tour.values()))
    origin_dest_km2={h: round(d[origin[h],destination[h]],2) for h in H if h not in Touren_neu}  
    gefahrene_km=sum(ges_km_je_Tour[h] for h in ges_km_je_Tour)+sum(origin_dest_km2[h] for h in origin_dest_km2)
    Externalitaet=gefahrene_km/sum(origin_dest_km[h] for h in H)
    return Touren_neu, Touren_sorted,used_origins,TW_LB_driver,TW_UB_driver, used_destinations, activitis, opt_S_driver,  Touren, active_nodes, Starting_times, Starting_times_formatted, active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, Externalitaet
result3 = variables_part2()
Touren_neu, Touren_sorted,used_origins,TW_LB_driver,TW_UB_driver, used_destinations, activitis, opt_S_driver,  Touren, active_nodes, Starting_times, Starting_times_formatted, active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, Externalitaet = result3 
"""
def to_excel():
    data_dict={"Modellname":"IPIC_SPp", "Stakeholder":"Plattform","WTP-Faktor":WTP_Faktor, "ETP-Faktor":ETP_Faktor, "DMD (Anzahl Aufträge)":len(A),"SPLDMD Level":float(m/n),"SPL(Anzahl verfügbarer Fahrer)":len(H), 
                "Anzahl Knoten gesamt":len(V), "Fahrer-Zeitfenster in min":ZF[0], "Laufzeit des Modells in sek":Runtime1, 
               "Optimalität (gap in %)":Optimalität, "Marge in €":round(OF_p,2),"Einnahmen in €":Einnahmen, "Kosten in €":Ausgaben,
               "Anzahl möglicher Matches":possible_matches , "Anzahl erfolgreicher Matches":matches, "Anzahl Touren":used_driver,"Marge je Tour":[Marge_je_Tour] ,"Touren":[Touren_sorted], 
               "WTP bearbeiteter Auftrtäge (€)":[used_WTP],"gewählter Preis je Auftrag (€)":[opt_request_p] , "gewählter Preis je km (€)":[chosen_p], "ETP eingesetzter Fahrer (€)":[used_ETP],
               "gewählte Kosten je km (€)":[chosen_cost], "Auftrag-Fahrer Zuordnung":[Auftrag_Fahrer], "Marge je Auftrag":[Marge_je_Auftrag], "Platform profit":OF_p, "Senders' surplus":Senders_surplus, 
               "Courier's surplus":Couriers_surplus,"flat_p":'', "flat_c":'', "Summe der Umwege":Umwege_summe, "Umweg je Tour":[Umweg], "gesamt km je Tour": [ges_km_je_Tour], "Umweg / ges.km": Umweg_ges_km,
               "Länge der ursprünglichen Fahrerstrecke":[origin_dest_km], "Umweg / Tourlänge je Tour":[Umweg_Tour_Verhältnis], "ETP je Fahrer": [ETP], "WTP je Kunde": [WTP], 
               "Streckenlänge je Auftrag": [km_Auftrag],"Verhältnis Umweg zur ursprünglichen Strecke je Tour":[Umweg_je_Tour], "Umweg/urprüngliche Strecke Schnitt": ave_Umweg_je_Tour, 
               "km ungematchter Fahrer":[origin_dest_km2], "gefahrene km insgesamt":gefahrene_km, "Verhältnis aller gefahrener km zu km ohne Aufträge":Externalitaet}
    df1=pd.DataFrame(data_dict)
    wb_row = load_workbook('C:/Users/rimbr/Desktop/final/Ergebnisse.xlsx')
    sheet=wb_row["ohne_ML"]
    neue_Zeile=len(sheet['B'])
    
    with pd.ExcelWriter('C:/Users/rimbr/Desktop/final/Ergebnisse.xlsx', mode='a', if_sheet_exists="overlay") as writer:
        df1.to_excel(writer, sheet_name="ohne_ML",header=False, index=False, startrow=neue_Zeile)
to_excel()
"""


#draw graphs
def graph_map():
    file_path = os.path.join(os.getcwd(), "data_Berlin", "berlin_ortsteile.shp")
    districts = gpd.read_file(file_path)
    G=nx.DiGraph()
    G.add_nodes_from(V)
    G.add_edges_from(active)
    pos={i: (AllX[i],AllY[i])for i in V}
    fig1 = plt.figure(figsize=(16, 10))
    graph_ax = fig1.add_subplot(1, 1, 1,projection=ccrs.PlateCarree())
    graph_ax=plt.axes([0,0,1,1], projection=ccrs.PlateCarree())
    graph_ax.add_feature(cfeature.LAKES, edgecolor='black', alpha=0.5)
    graph_ax.add_feature(cfeature.RIVERS, edgecolor='black', alpha=0.9)
    for i in district_names:
        graph_ax.text(district_coordy[i], district_coordx[i], district_names[i], fontsize=8, color='grey', ha='center', va='top')
    districts.boundary.plot(ax=graph_ax, color='gray',  linewidth=1, alpha=0.5)
    graph_ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_0_boundary_lines_land', scale='10m', facecolor=None))
    graph_ax.set_extent([13.27, 13.51, 52.450, 52.56], crs=ccrs.PlateCarree())
    file_path2 = os.path.join(os.getcwd(), "data_Berlin", "Umweltzone_Berlin.shp")
    Umweltzone = gpd.read_file(file_path2)
    target_crs = ccrs.PlateCarree()
    Umweltzone = Umweltzone.to_crs(target_crs.proj4_init)
    for idx, polygon in Umweltzone.iterrows():
        graph_ax.add_geometries([polygon['geometry']], crs=ccrs.PlateCarree(), facecolor='green', edgecolor='black', alpha=0.1)
    options = {"edgecolors": "k", "node_size": 200, "alpha": 0.5}
    nx.draw_networkx_nodes(G, pos, nodelist=tau[1:], node_color="yellow", node_shape='H', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A, node_color="tab:orange", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A_, node_color="tab:red", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=tau_[1:], node_color="tab:brown", node_shape='d', **options)
    nx.draw_networkx_labels(G, pos, labels={n:n for n in G}, font_size=8, font_color='k' )
    edges=list(G.edges())
    num_edges=len(edges)+1
    colormap=cm.get_cmap('tab20')
    colores=[colormap(i/(num_edges)) for i in range(max(Touren)+1)]
    edgecolors={(a):colores[h] for h in Touren for (a) in Touren[h]}
    driver_color={(h):colores[h] for h in active_driver}
    edge_color_list = [edgecolors[a] for a in edges]
    nx.draw_networkx_edges(G, pos, edge_color=edge_color_list, alpha=1.0 ,width=1.5, arrows=True, arrowsize=12, ax=graph_ax, connectionstyle="arc3,rad=0.1")
    graph_ax.set_xticks([])
    graph_ax.set_yticks([])
    graph_ax.set_xlim(13.27, 13.51)
    graph_ax.set_ylim(52.450, 52.562)
    graph_ax.gridlines(draw_labels=True)
    graph_ax.add_feature(cfeature.LAND)
    plt.title("Tourenmap:IPIC_SP_p")
    plt.show()
    
    return edgecolors, colores, num_edges, driver_color, fig1
result4=graph_map()
edgecolors, colores, num_edges, driver_color, fig1 = result4


def graph_gantt():    
    Ending_times = {}
    Ending_times = {i: (Starting_times[i] + s[i] + active_t[i]) for i in Starting_times if i not in tau_[1:]}
    Ending_times2 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    Ending_times.update(Ending_times2)
    Ending_times_formatted = {i: 0 for i in Starting_times}
    for key, value in Ending_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Ending_times_formatted[key] = time_str
    start_3={i:Starting_times[i] + s[i]  for i in Starting_times if i not in tau_[1:]}
    start_32 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    start_3.update(start_32)
    start3= {i: 0 for i in Starting_times}
    for key, value in start_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        start3[key] = time_str
    end_3={i:Starting_times[i] + s[i] + active_t[i]  for i in Starting_times if i not in tau_[1:]}
    end_32={i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    end_3.update(end_32)
    end3 = {i: 0 for i in Starting_times}
    for key, value in end_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        end3[key] = time_str
    df = []
    for h, i in Touren_sorted.items():
       for j in i:
            df.append({'task': j, 'Start': Starting_times_formatted[j], 'Finish': Ending_times_formatted[j], 'Resource': h, 'color': driver_color[h], 'start2':TW_LB_driver[h], 'end2':TW_UB_driver[h], 'start3':start3[j], 'end3':end3[j]})
    fig2, gantt_ax = plt.subplots(figsize=(16, 8))
    for item in df:
        start = datetime.strptime(item['Start'], '%H:%M')
        finish = datetime.strptime(item['Finish'], '%H:%M')
        duration = finish - start
        text_x2 = start+ duration/5
        text_y2 = item['Resource']
        gantt_ax.broken_barh([(start, duration)], (item['Resource'] - 0.25, 0.5), facecolors=item['color'], zorder=3, alpha=0.5)
        gantt_ax.annotate(item['task'], (text_x2, text_y2), xytext=(0, 0), textcoords='offset points',
                          ha='center', va='bottom', fontsize=9, color='black', zorder=5)
        start2 = datetime.strptime(item['start2'], '%H:%M')
        end2 = datetime.strptime(item['end2'], '%H:%M')
        start3= datetime.strptime(item['start3'], '%H:%M')
        end3 = datetime.strptime(item['end3'], '%H:%M')
        duration2 = end2 - start2
        duration3 = end3 - start3
        gantt_ax.broken_barh([(start2, duration2)], (item['Resource'] -0.25, 0.5), facecolors=item['color'], zorder=2, alpha=0.1)
        gantt_ax.broken_barh([(start3, duration3)], (item['Resource'] - 0.25, 0.5), facecolor='none', edgecolor='lightgrey', hatch='//', zorder=4)
    Tourdauer = mpatches.Patch(color='tab:red', label='Servicezeit am Knoten')
    Fahrer_ZF = mpatches.Patch(color='pink', label='Zeitfenster der Fahrer in hell')
    Fahrtzeit = mpatches.Patch(facecolor='none', hatch='//', edgecolor='grey', label='Fahrtzeit')
    Knoten_nummer = mpatches.Patch(color='white', label='1,2,.: Nr besuchter Knoten ')
    gantt_ax.legend(handles=[Tourdauer, Fahrer_ZF, Fahrtzeit, Knoten_nummer], loc='lower right')    
    plt.xlabel("Zeit")
    plt.ylabel("Fahrer")
    plt.title("Gantt:IPIC_SP_p")
    y_ticks = list(Touren_sorted.keys())
    y_labels = list(Touren_sorted.keys())  
    y_pos=list(Touren_sorted.keys())
    gantt_ax.set_yticks(y_pos, y_ticks)
    gantt_ax.set_yticklabels(y_labels) 
    x_ticks = pd.date_range(start=datetime(1900, 1, 1, 10, 0), end=datetime(1900, 1, 1, 20, 00), freq='H')
    x_labels = [time.strftime('%H:%M') for time in x_ticks]
    gantt_ax.set_xticks(x_ticks)
    gantt_ax.set_xticklabels(x_labels, rotation=45, ha='right')
    gantt_ax.grid(zorder=0)
    plt.show()
    #fig2.savefig('C:/Users/rimbr/Desktop/final/Vergleich ML ohne ML/12 tasks/ohne ML/IPIC SPp SPL1.0 WTP1ETP/Gantt_IPIC_SPp_2h_12_ohneML.png')
    return Ending_times, Ending_times_formatted, df, start3, fig2

result5 = graph_gantt()
Ending_times, Ending_times_formatted, df, start3, fig2 =result5


    
